# Fig 6: A holistic biopsychosocial framework for the development of chronic pain

### This notebook contains the analyses code for figure 6:
- Construct the structural equation model using biological (blood) and psyschosocial risk scores measured at baseline for rheumatoid arthritis (RA), the development of RA within 10 years after baseline, and the development of pain symptoms (impact, spreading, and interference) ~ 10 years after baseline
- This code shows the calculation of the structural equation model coefficients, however, the visualization was done using Biorender

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.stats import zscore
from scipy import stats
import warnings

# This will ignore all warnings
warnings.simplefilter(action='ignore', category=Warning)

line_width = 0.5
fs = 7

sns.set_context(rc={"font.size":fs})   
plt.rcParams["font.family"] = "Helvetica"
plt.rcParams['figure.dpi'] = 300
plt.rc('mathtext',**{'default':'regular'})
plt.rcParams['font.size'] = fs
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['xtick.major.size'] = 2
plt.rcParams['ytick.major.size'] = 2
plt.rcParams['xtick.major.pad'] = '1.5'
plt.rcParams['ytick.major.pad'] = '1.5'
plt.rcParams['axes.linewidth'] = line_width
plt.rcParams['xtick.major.width'] = line_width
plt.rcParams['ytick.major.width'] = line_width
# colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f']

color_mapping = {'Blood': '#9EB9F3',
 'Bone': '#F89C74',
 'PRS': '#DCB0F2',
 'Stacked': '#87C55F',
 'Psychosocial': '#C0BFC9'}

In [2]:
nci_cols_t0 = [
                    'NCI_fibromyalgia_T0',
                    'NCI_polymyalgia rheumatica_T0',
                    'NCI_cervical spondylosis_T0',
                    'NCI_joint pain_T0',
                    'NCI_back pain_T0',
                    'NCI_spine arthritis/spondylitis_T0',
                    'NCI_trapped nerve/compressed nerve_T0',
                    'NCI_sciatica_T0',
                    'NCI_hernia_T0',
                    'NCI_irritable bowel syndrome_T0',
                    'NCI_gastro-oesophageal reflux (gord) / gastric reflux_T0',
                    'NCI_arthritis (nos)_T0',
                    'NCI_osteoarthritis_T0',
                    'NCI_osteoporosis_T0',
                    'NCI_rheumatoid arthritis_T0',
                    'NCI_migraine_T0',
                    'NCI_headaches (not migraine)_T0',
                    'NCI_carpal tunnel syndrome_T0',
                    'NCI_angina_T0',
                    'NCI_endometriosis_T0',
                    'NCI_gout_T0',
                    'NCI_chronic fatigue syndrome_T0',
                    'NCI_ankylosing spondylitis_T0',
                    'NCI_trigemminal neuralgia_T0',
                    'NCI_crohns disease_T0',
                    'NCI_spinal stenosis_T0',
                    'NCI_peripheral neuropathy_T0',
                    'NCI_ulcerative colitis_T0',
                    'NCI_pulmonary embolism +/- dvt_T0',
                    'NCI_chronic obstructive airways disease/copd_T0',
                    'NCI_stroke_T0',
                    'NCI_multiple sclerosis_T0',
                    'NCI_psoriatic arthropathy_T0',
                    'NCI_Post Surgical Pain_T0',
                    # 'NCI_parkinsons disease_T0',
                    'NCI_disc disorder_T0',
                	# 'NCI_peripheral vascular disease_T0'
]

nci_cols_t1 = [i.replace('T0','T1') for i in nci_cols_t0]
nci_cols_t2 = [i.replace('T0','T2') for i in nci_cols_t0]
nci_cols_t3 = [i.replace('T0','T3') for i in nci_cols_t0]


In [3]:
diag_T0 = pd.read_pickle('/Users/Patty/Desktop/Biomarker_Paper/Pickled_Models/Diagnoses/first_occ/T0_diagnoses_clf-LR_lin.pickle')
diag_T0 = {key: value for key, value in diag_T0.items() if 'parkinson' not in key}
diag_T0 = {key: value for key, value in diag_T0.items() if 'peripheral vascular disease' not in key}

diag_T2 = pd.read_pickle('/Users/Patty/Desktop/Biomarker_Paper/Pickled_Models/Diagnoses/first_occ/T2_diagnoses_clf-LR_lin.pickle')
diag_T2 = {key: value for key, value in diag_T2.items() if 'parkinson' not in key}
diag_T2 = {key: value for key, value in diag_T2.items() if 'peripheral vascular disease' not in key}

In [4]:
def pivot_pickle(df,cols):
    
    T0_cat = pd.concat([df[i]['results_df'] for i in df.keys()])
    T0_cat.index = [i[:-4] for i in T0_cat.index]
    T0_auc = T0_cat.groupby(level=0).mean().AUC_test
    # T0_auc = T0_auc[~T0_auc.index.str.contains('Sparse')]
    ind = np.unique([i.split('_')[1:-1][0] for i in T0_auc.index])
    df = pd.concat([T0_auc[T0_auc.index.str.contains(i)].reset_index(drop=True) for i in cols],axis=1)
    df.index = ind
    df.columns = cols
    
    return df

T0_auc = pivot_pickle(diag_T0,cols=['Blood','PRS','Psychosocial'])
T2_auc = pivot_pickle(diag_T2,cols=['T1','DWI','fMRI','Stacked','Bone','Biology'])
merged_auc = pd.concat([T0_auc,T2_auc],axis=1).reindex([i.split('_')[1:-1][0] for i in nci_cols_t0])


In [5]:
home_dir = '/Users/Patty/Desktop/EVP_lab/'

cols_t0 = [
        'ChronicPain_T0',
        'ChronicWidespreadPain_T0', 
        'ChronicHeadaches_T0',
        'ChronicNeckShoulderPain_T0', 
        'ChronicHipPain_T0', 
        'ChronicBackPain_T0', 
        'ChronicStomachAbdominalPain_T0', 
        'ChronicKneePain_T0',
        'ChronicFacialPain_T0', 
        'NumberChronicPainTypes_T0',
        'Age_T0',
        'OverallHealthRating_T0',
        'UnableWorkDueSicknessDisability_T0'
]
cols_t1 = [i.replace('T0','T1') for i in cols_t0]
cols_t2 = [i.replace('T0','T2') for i in cols_t0]

cols = ['eid', 'Sex_T0'] + cols_t0 + cols_t1 + cols_t2
UKB = pd.read_csv(home_dir + 'UKB_NoBrain_500K_V4.csv',low_memory=False,usecols=cols) #UKB demographic data

# NCI Data
NCI_changes = pd.read_csv('/Users/Patty/Desktop/EVP_lab/NCI_changes.csv')

# # Adjust for WSP and calculate spreading
UKB.loc[UKB['ChronicWidespreadPain_T0'] == 1, 'NumberChronicPainTypes_T0'] = 7
UKB.loc[UKB['ChronicWidespreadPain_T1'] == 1, 'NumberChronicPainTypes_T1'] = 7
UKB.loc[UKB['ChronicWidespreadPain_T2'] == 1, 'NumberChronicPainTypes_T2'] = 7

UKB['Spread_T1'] = UKB['NumberChronicPainTypes_T1'] - UKB['NumberChronicPainTypes_T0']
UKB['Spread_T2'] = UKB['NumberChronicPainTypes_T2'] - UKB['NumberChronicPainTypes_T0']

UKB.loc[UKB['NumberChronicPainTypes_T0'] > 4, 'NumberChronicPainTypes_T0'] = 4
UKB.loc[UKB['NumberChronicPainTypes_T1'] > 4, 'NumberChronicPainTypes_T1'] = 4
UKB.loc[UKB['NumberChronicPainTypes_T2'] > 4, 'NumberChronicPainTypes_T2'] = 4


## Calculate of RA blood-based and psychosocial-based risk scores and use semopy to construct structural equation model

In [6]:
first_occ = pd.read_csv('/Users/Patty/Desktop/EVP_lab/first_occ_Dx_EPQ.csv')
EPQ = pd.read_csv('/Users/Patty/Desktop/EVP_lab/Experience_of_Pain_Processed.csv')
EPQ = EPQ[['eid','Duration', 'WorstPain', 'BPI', 'PHQ','SS', 'Dx_RheumatoidArthritis', 'NumberChronicPainSites']]

In [7]:
def merge_dataframe(dic, nci, mod1, mod2):
    a_data = [pd.DataFrame(dic[i]['roc_curve'].T, columns=['eid',f'y_true_{mod1}',f'{nci}_{mod1}']) for i in dic.keys() if f'{nci}_{mod1}' in i]
    a_data_o = pd.concat([pd.DataFrame(dic[i]['roc_curve_other'].T, columns=['eid',f'y_true_{mod1}',f'{nci}_{mod1}']) for i in dic.keys() if f'{nci}_{mod1}' in i])
    a = pd.concat([pd.concat(a_data),a_data_o]).groupby('eid').mean()
    a.columns = ['Dx',mod1]
    
    b_data = [pd.DataFrame(dic[i]['roc_curve'].T, columns=['eid',f'y_true_{mod2}',f'{nci}_Psychosocial']) for i in dic.keys() if f'{nci}_{mod2}' in i]
    b_data_o = pd.concat([pd.DataFrame(dic[i]['roc_curve_other'].T, columns=['eid',f'y_true_{mod2}',f'{nci}_{mod2}']) for i in dic.keys() if f'{nci}_{mod2}' in i])
    b = pd.concat([pd.concat(b_data),b_data_o]).groupby('eid').mean()
    b.columns = ['Dx',mod2]
    b = b.drop(columns='Dx')

    merged_df = a.merge(b, on='eid')
    merged_df[mod1] = np.log(merged_df[mod1] / (1 - merged_df[mod1]))
    merged_df[mod2] = np.log(merged_df[mod2] / (1 - merged_df[mod2]))
    merged_df.columns = [nci,nci+'_'+mod1,nci+'_'+mod2] # Adds individual dx name to columns

    return merged_df

def create_pivot_heatmap(merged_auc, dic, UKB, mod, T, strata, n):
    
    # Extract the top n indices based on the specified modality
    n = merged_auc.sort_values(mod, ascending=False).index[:n]
    print(n)
    
    combined_df = pd.DataFrame()
    ncis = []
    for nci in n:
        tmp = merge_dataframe(dic, nci, mod, 'Psychosocial')
        combined_df = pd.concat([combined_df, tmp],axis=1)
        ncis.append(f'NCI_{nci}_{T}')
    
    return combined_df

TP = 'T0'
combined_sigs_blood = create_pivot_heatmap(merged_auc, diag_T0, UKB, 'Blood', TP, 5, 13)

Index(['gout', 'polymyalgia rheumatica', 'stroke', 'rheumatoid arthritis',
       'crohns disease', 'angina', 'psoriatic arthropathy',
       'peripheral neuropathy', 'ankylosing spondylitis',
       'carpal tunnel syndrome', 'pulmonary embolism +/- dvt',
       'ulcerative colitis', 'arthritis (nos)'],
      dtype='object')


In [8]:
first_occ.columns = [i.replace(' ','_') for i in first_occ.columns]
combined_sigs_blood.columns = [i.replace(' ','_') for i in combined_sigs_blood.columns]

In [13]:
first_occ['rheumatoid_arthritis_developed_EPQ'] = np.where(
    (first_occ['NCI_rheumatoid_arthritis_EPQ'] == 1) & (first_occ['NCI_rheumatoid_arthritis_T0'] == 0), 1, 0
)

tmp = first_occ.merge(EPQ,on='eid').merge(combined_sigs_blood[['rheumatoid_arthritis_Blood','rheumatoid_arthritis_Psychosocial']],on='eid')[
    ['eid','rheumatoid_arthritis_Blood','rheumatoid_arthritis_Psychosocial','rheumatoid_arthritis_developed_EPQ','NCI_EPQ', 'BPI', 'WorstPain', 'NumberChronicPainSites']
]
tmp = tmp[(tmp['rheumatoid_arthritis_developed_EPQ'] == 1) | (tmp['NCI_EPQ'] == 0)]

tmp = tmp.dropna()

# zscore
tmp[['rheumatoid_arthritis_developed_EPQ', 'BPI', 'WorstPain', 'NumberChronicPainSites']] = zscore(tmp[['rheumatoid_arthritis_developed_EPQ', 'BPI', 'WorstPain', 'NumberChronicPainSites']])
tmp[['rheumatoid_arthritis_Blood','rheumatoid_arthritis_Psychosocial']] = zscore(tmp[['rheumatoid_arthritis_Blood','rheumatoid_arthritis_Psychosocial']])
tmp['interaction'] = tmp.rheumatoid_arthritis_Blood * tmp.rheumatoid_arthritis_Psychosocial


import semopy
from semopy import Model


# BPI ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
# NumberChronicPainSites ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
# Duration ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
# PHQ ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
# WorstPain ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
# SS ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
    
model_spec = """
    rheumatoid_arthritis_developed_EPQ ~ rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial + interaction
    NumberChronicPainSites ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
    BPI ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
    WorstPain ~ rheumatoid_arthritis_developed_EPQ + rheumatoid_arthritis_Blood + rheumatoid_arthritis_Psychosocial
    """


# Initialize the Model using the specification
model = Model(model_spec)
model.fit(tmp)
print(model.inspect())

# # To visualize:
g = semopy.semplot(model, "/Users/Patty/Desktop/SEM_sigs.png")

stats = semopy.calc_stats(model)

                                  lval  op  \
0   rheumatoid_arthritis_developed_EPQ   ~   
1   rheumatoid_arthritis_developed_EPQ   ~   
2   rheumatoid_arthritis_developed_EPQ   ~   
3               NumberChronicPainSites   ~   
4               NumberChronicPainSites   ~   
5               NumberChronicPainSites   ~   
6                                  BPI   ~   
7                                  BPI   ~   
8                                  BPI   ~   
9                            WorstPain   ~   
10                           WorstPain   ~   
11                           WorstPain   ~   
12  rheumatoid_arthritis_developed_EPQ  ~~   
13                                 BPI  ~~   
14              NumberChronicPainSites  ~~   
15                           WorstPain  ~~   

                                  rval  Estimate  Std. Err    z-value  \
0           rheumatoid_arthritis_Blood  0.157752  0.015575  10.128420   
1    rheumatoid_arthritis_Psychosocial  0.227616  0.015770  14.433640  